In [2]:
import pandas as pd
import numpy as np

1. Load the original CSV

In [3]:
df = pd.read_csv("wonka_choc_factory.csv")

print(">>> Original shape:", df.shape)
print(df.head())

>>> Original shape: (10194, 19)
   Unnamed: 0  Order Date   Ship Date       Ship Mode  Customer ID  \
0           0  2021-03-31  2026-09-26  Standard Class       128055   
1           1  2021-03-31  2026-09-26  Standard Class       128055   
2           2  2021-09-15  2027-03-13  Standard Class       138100   
3           3  2021-09-15  2027-03-13  Standard Class       138100   
4           4  2022-10-04  2028-03-29     First Class       121391   

  Country/Region           City State/Province Postal Code   Division  \
0  United States  San Francisco     California       94122  Chocolate   
1  United States  San Francisco     California       94122  Chocolate   
2  United States  New York City       New York       10011  Chocolate   
3  United States  New York City       New York       10011  Chocolate   
4  United States  San Francisco     California       94109  Chocolate   

     Region                       Product Name  Sales  Units  Gross Profit  \
0   Pacific  Wonka Bar - Tripl

2. Drop unnecessary columns (e.g., exported index fields)

In [4]:
cols_to_drop = [col for col in df.columns if col.startswith("Unnamed")]
df = df.drop(columns=cols_to_drop)

3. Convert data types: dates and numeric fields

In [5]:
# Convert date columns to datetime
date_cols = ["Order Date", "Ship Date"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Convert numeric columns
num_cols = ["Sales", "Units", "Gross Profit", "Cost", "Latitude", "Longitude"]
for col in num_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

4. Handle missing values

In [6]:
text_cols = [
    "Ship Mode", "Customer Id", "Country/Region", "City", "State/Province",
    "Postal Code", "Division", "Region", "Product Name", "Factory"
]
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

# Numeric columns:
# Units → fill with median to avoid distortion
if "Units" in df.columns:
    df["Units"] = df["Units"].fillna(df["Units"].median())

# Sales, Gross Profit, Cost → fill with 0 (conservative approach)
for col in ["Sales", "Gross Profit", "Cost"]:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Latitude/Longitude → keep NaN (usually best for later geolocation inspection)


5. Create useful derived features

In [7]:
# Shipping delay in days
if {"Order Date", "Ship Date"}.issubset(df.columns):
    df["Shipping_Delay_Days"] = (df["Ship Date"] - df["Order Date"]).dt.days

# Profit margin
if {"Gross Profit", "Sales"}.issubset(df.columns):
    df["Profit_Margin"] = np.where(
        df["Sales"] != 0,
        df["Gross Profit"] / df["Sales"],
        np.nan
    )

# Price per unit
if {"Sales", "Units"}.issubset(df.columns):
    df["Price_per_Unit"] = np.where(
        df["Units"] != 0,
        df["Sales"] / df["Units"],
        np.nan
    )

# Cost per unit
if {"Cost", "Units"}.issubset(df.columns):
    df["Cost_per_Unit"] = np.where(
        df["Units"] != 0,
        df["Cost"] / df["Units"],
        np.nan
    )



6. Normalize text fields (clean formatting)

In [8]:
# - Remove extra spaces
# - Standardize case formatting (Title Case)

for col in text_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
            .str.title()
        )

# Postal code → always as string (avoid losing leading zeros)
if "Postal Code" in df.columns:
    df["Postal Code"] = df["Postal Code"].astype(str).str.strip()

7 Basic outlier detection (flagging only)

In [9]:
# Units outliers based on IQR rule
if "Units" in df.columns:
    q1 = df["Units"].quantile(0.25)
    q3 = df["Units"].quantile(0.75)
    iqr = q3 - q1
    lower = max(q1 - 1.5 * iqr, 0)
    upper = q3 + 1.5 * iqr
    
    df["Units_Outlier"] = (df["Units"] < lower) | (df["Units"] > upper)

# Profit margin outliers (negative or extremely high margins)
if "Profit_Margin" in df.columns:
    df["Profit_Margin_Outlier"] = (
        (df["Profit_Margin"] < 0) | (df["Profit_Margin"] > 1.5)
    )

8 Final dataset overview

In [10]:
# Select a preview of the dataset
preview = df.head(10)

preview

,Order Date,Ship Date,Ship Mode,Customer ID,Country/Region,City,State/Province,Postal Code,Division,Region,...,Cost,Factory,Latitude,Longitude,Shipping_Delay_Days,Profit_Margin,Price_per_Unit,Cost_per_Unit,Units_Outlier,Profit_Margin_Outlier
0,2021-03-31,2026-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,...,2.60,Wicked Choccy'S,32.076176,-81.088371,2005,0.653333,3.75,1.30,False,False
1,2021-03-31,2026-09-26,Standard Class,128055,United States,San Francisco,California,94122,Chocolate,Pacific,...,2.20,Lot'S O' Nuts,32.881893,-111.768036,2005,0.694444,3.60,1.10,False,False
2,2021-09-15,2027-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,...,2.40,Lot'S O' Nuts,32.881893,-111.768036,2005,0.666667,3.60,1.20,False,False
3,2021-09-15,2027-03-13,Standard Class,138100,United States,New York City,New York,10011,Chocolate,Atlantic,...,3.42,Wicked Choccy'S,32.076176,-81.088371,2005,0.649231,3.25,1.14,False,False
4,2022-10-04,2028-03-29,First Class,121391,United States,San Francisco,California,94109,Chocolate,Pacific,...,2.28,Wicked Choccy'S,32.076176,-81.088371,2003,0.649231,3.25,1.14,False,False
5,2023-03-03,2028-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,...,1.10,Lot'S O' Nuts,32.881893,-111.768036,2005,0.694444,3.60,1.10,False,False
6,2023-03-03,2028-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,...,3.00,Lot'S O' Nuts,32.881893,-111.768036,2005,0.713467,3.49,1.00,False,False
7,2023-03-03,2028-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,...,8.40,Lot'S O' Nuts,32.881893,-111.768036,2005,0.666667,3.60,1.20,False,False
8,2023-03-03,2028-08-28,Standard Class,103982,United States,Round Rock,Texas,78664,Chocolate,Interior,...,3.42,Wicked Choccy'S,32.076176,-81.088371,2005,0.649231,3.25,1.14,False,False
9,2024-06-29,2029-12-25,Standard Class,147039,United States,Minneapolis,Minnesota,55407,Chocolate,Interior,...,3.00,Lot'S O' Nuts,32.881893,-111.768036,2005,0.713467,3.49,1.00,False,False


In [16]:
df.shape

(10194, 24)

9. Save cleaned datase

In [11]:
df.to_csv("wonka_choc_factory_clean.csv", index=False)
print(">>> Clean CSV saved as 'wonka_choc_factory_clean.csv'")

>>> Clean CSV saved as 'wonka_choc_factory_clean.csv'
